In [27]:
# Your API key (on the e-mail)
# Andre: 4eadeb76-03d6-40cc-b858-27b0da441ac5
# Pedro: 175accd9-4b3d-42a7-998e-85600fa3d1f4
# Pierre: 7a1ed46e-6371-445e-a383-cc6ccf0525fd
API_KEY = "175accd9-4b3d-42a7-998e-85600fa3d1f4" 

# Your sciper number. 
# Andre: 376762
# Pedro: 374339
# Pierre: 385070
SCIPER = 374339 


# To actually rank the preference pairs, or to not save anything (so we can e.g. adjust the prompt we pass to ChatGPT)
playground = True

import gpt_wrapper
gpt_wrapper.api_base = "http://mnlp-backend-938795011.eu-central-1.elb.amazonaws.com"
gpt_wrapper.api_key = API_KEY

from gpt_wrapper.chat import Chat

In [76]:
import json
import random

# Open json file
questions = json.load(open(f"data/{SCIPER}_base.json"))
progress_questions = json.load(open(f"data/{SCIPER}.json"))

# Save a backup just in case
with open(f"data/{SCIPER}_bak.json", "w") as f:
    json.dump(progress_questions, f)

num_questions = len(questions)
num_progress = len(progress_questions)

print(f"Howdy! :D")
print(f"Missing {num_questions - num_progress} questions!")



Howdy! :D
Missing 100 questions!


In [69]:
print(questions[0])

{'course_id': 3, 'question_id': 6549, 'question_body': 'A particle of mass $m$ is subjected to a harmonic potential \r\n\\begin{equation}\r\n\\hat{V}= \\frac{1}{2}m\\omega^2 \\hat{x^2}\r\n\\end{equation}\r\nand, at time $t=0$ it is in a $|\\psi(t=0)\\rangle$ state determined by the following conditions:\r\n\\begin{enumerate}\r\n\\item each energy measurement gives certain values $E$ which satisfy the relation $ \\hbar \\omega < E < 3\\hbar \\omega $\r\n\\item the mean value of the energy is $ \\langle E \\rangle = \\frac{11}{6} \\hbar \\omega $\r\n\\item the mean value of the position is $ \\langle x \\rangle = -\\sqrt{\\frac{8\\hbar}{9m\\omega}} $\r\n\\end{enumerate}\r\nDetermine all the times $t>0$ when the average value of the position is positive and maximum. \r\n[Tip: in what follows, it may be useful to express the position operator in terms of the creation and destruction operators $\\hat{a}$ and $\\hat{a^{\\dagger}}$].', 'question_options': None}


In [20]:
chat1 = Chat.create("Test Chat")


In [24]:
print(chat1.ask("What did I ask?"))

You asked who won the FIFA championship in 2018.


In [99]:
# Create the template for the prompt that will be passed to ChatGPT
# Leave the <question> tag where the question should be inserted, as it will later be replaced by the actual question
# TODO: make two templates for open-ended and multi-choice questions
# TODO: read the suggested papers
# TODO: play with the templates to find a good one

template_open = """
Let's think step-by-step. Please answer the following question:
<question>
"""

template_mcq = """
Let's think step-by-step. Please answer the following question:
<question>
"""

# Set the specific question you want to ask
if playground:
    num_progress = 6

In [100]:
while num_progress < num_questions:

    if playground:
        chatA = Chat.create("Playground chat A " + str(random.randint(0, 10000000000)))
        chatB = Chat.create("Playground chat B " + str(random.randint(0, 10000000000)))
    else:
        chatA = Chat.create("Chat A for question " + str(num_progress) + " " + str(random.randint(0, 10000000000)))
        chatB = Chat.create("Chat B for question " + str(num_progress) + " " + str(random.randint(0, 10000000000)))
    
    template = template_open
    
    questionData = questions[num_progress]
    
    question = questionData["question_body"]
    if questionData["question_options"] != None:
        # if it's a multiple choice question, change the template
        template = template_mcq
        
        options = questionData["question_options"]
        for i, option in enumerate(options):
            question += f"\n{chr(ord('A') + i)}: {option}"
    
    toAsk = template.replace("<question>", question)
    
    responseA = chatA.ask(toAsk)
    responseB = chatB.ask(toAsk)
    A = str(responseA)
    B = str(responseB)
    
    # Depending on the passed template, change this so that it extracts the answer from the response accordingly
    # A_start = response_str.find("<A_START>") + len("<A_START>")
    # A_end = response_str.find("<A_END>")
    # B_start = response_str.find("<B_START>") + len("<B_START>")
    # B_end = response_str.find("<B_END>")
    
    # A = response_str[A_start:A_end].split("A: ")[1].strip()
    # B = response_str[B_start:B_end].split("B: ")[1].strip()

    print("Asked:")
    print(toAsk)
    print("--------------")
    print("Answer A:")
    print(A)
    print("--------------")
    print("Answer B:")
    print(B)
    
    overall, correctness, relevance, clarity, completeness, other = -1, -1, -1, -1, -1, -1
    while overall not in ["A", "B", "AB", "N"]:
        overall = input("Overall ranking (A, B, AB, N for None): ")
    while correctness not in ["A", "B", "AB", "N"]:
        correctness = input("Correctness ranking (A, B, AB, N): ")
    while relevance not in ["A", "B", "AB", "N"]:
        relevance = input("Relevance ranking (A, B, AB, N): ")
    while clarity not in ["A", "B", "AB", "N"]:
        clarity = input("Clarity ranking (A, B, AB, N): ")
    while completeness not in ["A", "B", "AB", "N"]:
        completeness = input("Completeness ranking (A, B, AB, N): ")
    
    other = input("Other ranking (None or `<criteria1>: <A, B, AB>;<criteria2>: <A, B, AB>;...`): ")
    
    if overall == "N":
        overall = "None"
    if correctness == "N":
        correctness = "None"
    if relevance == "N":
        relevance = "None"
    if clarity == "N":
        clarity = "None"
    if completeness == "N":
        completeness = "None"
    if other == "N" or other == "":
        other = "None"
    
    
    progress_questions += [{
        "course_id": questionData["course_id"],
        "question_id": questionData["question_id"],
        "question": questionData["question_body"],
        
        "A_chat_id": responseA.chat_id,
        "B_chat_id": responseB.chat_id,
        
        "A": A, 
        "B": B,
        
        "ranking_criteria": {
            "overall": overall,  
            "correctness": correctness,  
            "relevance": relevance,  
            "clarity": clarity,  
            "completeness": completeness,  
            "other": other,  
        }
    }] 
    
    # Saves first to a backup file, then to the actual file
    if not playground:
        with open(f"data/{SCIPER}_bak.json", "w") as f:
            json.dump(progress_questions, f)
            
        with open(f"data/{SCIPER}.json", "w") as f:
            json.dump(progress_questions, f)

        num_progress += 1
        
    break

Asked:

Let's think step-by-step. Please answer the following question:
Suppose we have the binary plaintext $0011$ and we encrypt it with the Vernam cipher and the binary key $0101$. The ciphertext is\dots
A: $0110$.
B: $1000$.
C: $0111$.
D: $0001$.

--------------
Answer A:
To encrypt using the Vernam cipher, we perform a XOR operation between each bit of the plaintext and the corresponding bit of the key.

Plaintext: $0011$
Key: $0101$

Performing XOR operation:
$0 \oplus 0 = 0$
$0 \oplus 1 = 1$
$1 \oplus 0 = 1$
$1 \oplus 1 = 0$

So the ciphertext would be $0110$, which corresponds to option A.
--------------
Answer B:
To encrypt using the Vernam cipher, we perform a bitwise XOR operation between the plaintext and the key.

Given:
Plaintext: $0011$
Key: $0101$

Performing the XOR operation:
$0011 \oplus 0101 = 0110$

Therefore, the ciphertext is $0110$, so the correct answer is A.
